In [12]:
# this is my first shoot in loan Pred competition

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [59]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [75]:
for col in train.columns:
    
    print(col,sum(train[col].isnull()))
    

Loan_ID 0
Gender 13
Married 3
Dependents 11
Education 0
Self_Employed 32
ApplicantIncome 0
CoapplicantIncome 0
LoanAmount 22
Loan_Amount_Term 14
Credit_History 50
Property_Area 0
Loan_Status 0


In [19]:
# replace NA LoanAmount with mean
train.loc[train.LoanAmount.isnull(),'LoanAmount']=np.mean(train.loc[train.LoanAmount.notnull(),
                                                                    'LoanAmount'] ) 

In [76]:
train[train.Dependents.isnull()]

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
102,LP001350,Male,Yes,NaN,Graduate,No,13650,0.0,NaN,360.0,1.0,Urban,Y
104,LP001357,Male,NaN,NaN,Graduate,No,3816,754.0,160.0,360.0,1.0,Urban,Y
120,LP001426,Male,Yes,NaN,Graduate,No,5667,2667.0,180.0,360.0,1.0,Rural,Y
226,LP001754,Male,Yes,NaN,Not Graduate,Yes,4735,0.0,138.0,360.0,1.0,Urban,N
228,LP001760,Male,NaN,NaN,Graduate,No,4758,0.0,158.0,480.0,1.0,Semiurban,Y
301,LP001972,Male,Yes,NaN,Not Graduate,No,2875,1750.0,105.0,360.0,1.0,Semiurban,Y
335,LP002106,Male,Yes,NaN,Graduate,Yes,5503,4490.0,70.0,NaN,1.0,Semiurban,Y
346,LP002130,Male,Yes,NaN,Not Graduate,No,3523,3230.0,152.0,360.0,0.0,Rural,N
435,LP002393,Female,NaN,NaN,Graduate,No,10047,0.0,NaN,240.0,1.0,Semiurban,Y
517,LP002682,Male,Yes,NaN,Not Graduate,No,3074,1800.0,123.0,360.0,0.0,Semiurban,N


In [74]:
# not married have not independent
train.loc[(train.Dependents.isnull()) & (  train.Married=='No'),'Dependents']=0

In [55]:

train.loc[train.Gender.isnull(),'Gender']='Male'
train.loc[train.Married.isnull(),'Married']='Yes'
train.loc[train.Credit_History.isnull(),'Credit_History']=1.0
train.loc[train.Loan_Amount_Term.isnull(),'Loan_Amount_Term']=360
train.loc[train.Self_Employed.isnull(),'Self_Employed']='No'


In [57]:
pd.value_counts(train.Dependents)

0     345
1     102
2     101
3+     51
Name: Dependents, dtype: int64